# 1. Import Dependencies

In [50]:
import pandas as pd
import os
import plotly.express as px
import plotly.graph_objects as go

Fetch the data from the **COVID-19 Data Repository by the Center for Systems Science and Engineering (CSSE) at Johns Hopkins University**

In [8]:
cases= pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')

In [4]:
deaths= pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')

# 2. Do Preprocessing on the Cases Data

In [9]:
cases.shape

(274, 454)

In [10]:
cases

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/6/21,4/7/21,4/8/21,4/9/21,4/10/21,4/11/21,4/12/21,4/13/21,4/14/21,4/15/21
0,NaN,Afghanistan,33.939110,67.709953,0,0,0,0,0,0,...,56779,56873,56943,57019,57144,57160,57242,57364,57492,57534
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,...,126936,127192,127509,127795,128155,128393,128518,128752,128959,129128
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,...,117879,118004,118116,118251,118378,118516,118645,118799,118975,119142
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,...,12328,12363,12409,12456,12497,12545,12581,12614,12641,12641
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,...,22885,23010,23108,23242,23331,23457,23549,23697,23841,23951
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269,NaN,Vietnam,14.058324,108.277199,0,2,2,2,2,2,...,2648,2659,2668,2683,2692,2693,2705,2714,2733,2758
270,NaN,West Bank and Gaza,31.952200,35.233200,0,0,0,0,0,0,...,256461,259133,262017,264395,265897,268132,270856,272767,274690,276407
271,NaN,Yemen,15.552727,48.516388,0,0,0,0,0,0,...,4975,5047,5133,5233,5276,5357,5446,5507,5582,5657
272,NaN,Zambia,-13.133897,27.849332,0,0,0,0,0,0,...,89071,89386,89592,89783,89918,90029,90064,90218,90389,90532


In [11]:
countries=cases["Country/Region"].count()
unique_countries=cases["Country/Region"].nunique(dropna = True)
print("There are ",countries ,"entries for the country column while unique no of countries is ",unique_countries)

There are  274 entries for the country column while unique no of countries is  192


This means there are repeated countries due to the seperate divisions for some countries inthe Province/State column like the US and Australia 

In [14]:
cases["Province/State"].nunique(dropna = True)

85

##### Drop the Province/State column, then group all rows by country.

In [16]:
# df1 shall be used as the dataframe to do preprocessing on
df1=cases.drop('Province/State',axis=1)
df1.shape

(274, 453)

In [17]:
df1["Country/Region"].nunique(dropna = True)

192

##### We have to obtain a similar number of rows to the one in the above output after grouping by country/Region

In [20]:
df1=df1.groupby(['Country/Region'],as_index=False).sum()
df1.shape

(192, 453)

##### Great, they match up, the groupby operation was successful

In [21]:
df1.head()

,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,...,4/6/21,4/7/21,4/8/21,4/9/21,4/10/21,4/11/21,4/12/21,4/13/21,4/14/21,4/15/21
0,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,0,...,56779,56873,56943,57019,57144,57160,57242,57364,57492,57534
1,Albania,41.15330,20.168300,0,0,0,0,0,0,0,...,126936,127192,127509,127795,128155,128393,128518,128752,128959,129128
2,Algeria,28.03390,1.659600,0,0,0,0,0,0,0,...,117879,118004,118116,118251,118378,118516,118645,118799,118975,119142
3,Andorra,42.50630,1.521800,0,0,0,0,0,0,0,...,12328,12363,12409,12456,12497,12545,12581,12614,12641,12641
4,Angola,-11.20270,17.873900,0,0,0,0,0,0,0,...,22885,23010,23108,23242,23331,23457,23549,23697,23841,23951


##### For the project, we don't need the Latitude and Longitude columns of our data.

In [22]:
df1=df1.drop(['Lat','Long'],axis=1)    

##### Obtain the Country codes from the Plotly Express documentation.Theyre useful in plotting the choropleth map

In [23]:
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2014_world_gdp_with_codes.csv')
df

,COUNTRY,GDP (BILLIONS),CODE
0,Afghanistan,21.71,AFG
1,Albania,13.40,ALB
2,Algeria,227.80,DZA
3,American Samoa,0.75,ASM
4,Andorra,4.80,AND
...,...,...,...
217,Virgin Islands,5.08,VGB
218,West Bank,6.64,WBG
219,Yemen,45.45,YEM
220,Zambia,25.61,ZMB


##### Drop the GDP column since we do not need it.

In [24]:
df=df.drop('GDP (BILLIONS)',axis=1)
df

,COUNTRY,CODE
0,Afghanistan,AFG
1,Albania,ALB
2,Algeria,DZA
3,American Samoa,ASM
4,Andorra,AND
...,...,...
217,Virgin Islands,VGB
218,West Bank,WBG
219,Yemen,YEM
220,Zambia,ZMB


In [25]:
df1

,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,...,4/6/21,4/7/21,4/8/21,4/9/21,4/10/21,4/11/21,4/12/21,4/13/21,4/14/21,4/15/21
0,Afghanistan,0,0,0,0,0,0,0,0,0,...,56779,56873,56943,57019,57144,57160,57242,57364,57492,57534
1,Albania,0,0,0,0,0,0,0,0,0,...,126936,127192,127509,127795,128155,128393,128518,128752,128959,129128
2,Algeria,0,0,0,0,0,0,0,0,0,...,117879,118004,118116,118251,118378,118516,118645,118799,118975,119142
3,Andorra,0,0,0,0,0,0,0,0,0,...,12328,12363,12409,12456,12497,12545,12581,12614,12641,12641
4,Angola,0,0,0,0,0,0,0,0,0,...,22885,23010,23108,23242,23331,23457,23549,23697,23841,23951
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,Vietnam,0,2,2,2,2,2,2,2,2,...,2648,2659,2668,2683,2692,2693,2705,2714,2733,2758
188,West Bank and Gaza,0,0,0,0,0,0,0,0,0,...,256461,259133,262017,264395,265897,268132,270856,272767,274690,276407
189,Yemen,0,0,0,0,0,0,0,0,0,...,4975,5047,5133,5233,5276,5357,5446,5507,5582,5657
190,Zambia,0,0,0,0,0,0,0,0,0,...,89071,89386,89592,89783,89918,90029,90064,90218,90389,90532


##### Drop all columns from the first date to the second last.
Since we need only the total cases on the last day for our map.

In [27]:
cols=df1[df1.columns[1:-1]]
df2=df1.drop(cols,axis=1)

In [29]:
#df2 will be the new sliced dataframe 
df2

,Country/Region,4/15/21
0,Afghanistan,57534
1,Albania,129128
2,Algeria,119142
3,Andorra,12641
4,Angola,23951
...,...,...
187,Vietnam,2758
188,West Bank and Gaza,276407
189,Yemen,5657
190,Zambia,90532


##### DO a left join on the Country/Region and COUNTRY columns

In [30]:
df3=pd.merge(df2,df,how='left',left_on=['Country/Region'],right_on=['COUNTRY'])
df3

,Country/Region,4/15/21,COUNTRY,CODE
0,Afghanistan,57534,Afghanistan,AFG
1,Albania,129128,Albania,ALB
2,Algeria,119142,Algeria,DZA
3,Andorra,12641,Andorra,AND
4,Angola,23951,Angola,AGO
...,...,...,...,...
187,Vietnam,2758,Vietnam,VNM
188,West Bank and Gaza,276407,NaN,NaN
189,Yemen,5657,Yemen,YEM
190,Zambia,90532,Zambia,ZMB


##### Drop the COUNTRY column from the new dataframe (df3)

In [31]:
df3=df3.drop('COUNTRY',axis=1)
df3

,Country/Region,4/15/21,CODE
0,Afghanistan,57534,AFG
1,Albania,129128,ALB
2,Algeria,119142,DZA
3,Andorra,12641,AND
4,Angola,23951,AGO
...,...,...,...
187,Vietnam,2758,VNM
188,West Bank and Gaza,276407,NaN
189,Yemen,5657,YEM
190,Zambia,90532,ZMB


##### Check whether there are any missing values due to mismatching Country names in the two datasets and missing codes 
##### for some countries in the left dataset.

In [32]:
df3['CODE'].isnull().values.any()

True

In [74]:
empty= df3[df3.isna().any(axis=1)]
empty

,Country/Region,4/15/21,CODE
11,Bahamas,9505,NaN
39,Congo (Brazzaville),10084,NaN
40,Congo (Kinshasa),28769,NaN
46,Czechia,1593847,NaN
48,Diamond Princess,712,NaN
58,Eswatini,18412,NaN
64,Gambia,5694,NaN
75,Holy See,27,NaN
104,MS Zaandam,9,NaN
115,Micronesia,1,NaN


##### Save the plotly map codes to a csv, edit it with all these missing codes, re-read it and do an inner join using the final version.

In [75]:
# df.to_excel ("map_code_copy.xlsx", index = False, header=True)

In [76]:
import os
pwd=os.getcwd()
df4=pd.read_excel(pwd+"\\map_code_copy.xlsx")

In [77]:
df4

,COUNTRY,CODE
0,Afghanistan,AFG
1,Albania,ALB
2,Algeria,DZA
3,American Samoa,ASM
4,Andorra,AND
...,...,...
217,Virgin Islands,VGB
218,West Bank and Gaza,WBG
219,Yemen,YEM
220,Zambia,ZMB


##### Save it as a new file for use in final script

In [78]:
df4.to_excel ("map_code1.xlsx", index = False, header=True)

##### Perform inner join on the map code and cases(df2)
This step eliminates countries (Holy See,North Macedonia and Micronesia) and cruise ships (Diamond Princess and MS Zaandam)in the cases dataset.

In [79]:
df5=pd.merge(df2,df4,how='inner',left_on=['Country/Region'],right_on=['COUNTRY'])
df5=df5.drop('COUNTRY',axis=1)
df5

,Country/Region,4/15/21,CODE
0,Afghanistan,57534,AFG
1,Albania,129128,ALB
2,Algeria,119142,DZA
3,Andorra,12641,AND
4,Angola,23951,AGO
...,...,...,...
182,Vietnam,2758,VNM
183,West Bank and Gaza,276407,WBG
184,Yemen,5657,YEM
185,Zambia,90532,ZMB


In [80]:
# Check for missing values
empty= df5[df5.isna().any(axis=1)]
empty

,Country/Region,4/15/21,CODE


In [81]:
# Rename the Cases column
casemapdata = df5.rename(columns={df5.columns[1]: 'Total Cases'})
casemapdata

,Country/Region,Total Cases,CODE
0,Afghanistan,57534,AFG
1,Albania,129128,ALB
2,Algeria,119142,DZA
3,Andorra,12641,AND
4,Angola,23951,AGO
...,...,...,...
182,Vietnam,2758,VNM
183,West Bank and Gaza,276407,WBG
184,Yemen,5657,YEM
185,Zambia,90532,ZMB


# 3. Fit the data to the Plotly Choropleth map

In [86]:

fig = go.Figure(data=go.Choropleth(
    locations = casemapdata['CODE'],
    z = casemapdata['Total Cases'],
    text = casemapdata['Country/Region'],
    colorscale = 'Blues',
    autocolorscale=False,
    reversescale=True,
    marker_line_color='darkgray',
    marker_line_width=0.5,
    colorbar_title = 'Total Cases',
))

fig.update_layout(
    title_text='Cumulative Cases per Country',
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    ),
    annotations = [dict(
        x=0.55,
        y=0.1,
        xref='paper',
        yref='paper',
        text='Source: <a href="https://github.com/CSSEGISandData/COVID-19">\
            JHU CSSE COVID-19 Data</a>',
        showarrow = False
    )]
)

fig.show()

In [84]:
casemapdata.to_excel ("casemapdata.xlsx", index = False, header=True)

# 4. Do Preprocessing on the Deaths Data

In [87]:
deaths.shape

(274, 454)

##### Drop the Province/State column, then group all rows by country.
##### df1 shall be used as the dataframe to do preprocessing on

In [93]:
df6=deaths.drop('Province/State',axis=1)
df6.shape

(274, 453)

In [94]:
df6["Country/Region"].nunique(dropna = True)

192

###### We have to obtain a similar number of rows to the one in the above output after grouping by country/Region

In [97]:
df6=df6.groupby(['Country/Region'],as_index=False).sum()
df6.shape

(192, 453)

##### Great, they match up, the groupby operation was successful

##### For the project, we don't need the Latitude and Longitude columns of our data.   

In [98]:
df6=df6.drop(['Lat','Long'],axis=1)    

In [99]:
cols2=df6[df6.columns[1:-1]]
df7=df6.drop(cols2,axis=1)
#df7 will be the new sliced dataframe 
df7

,Country/Region,4/15/21
0,Afghanistan,2533
1,Albania,2335
2,Algeria,3144
3,Andorra,121
4,Angola,557
...,...,...
187,Vietnam,35
188,West Bank and Gaza,2937
189,Yemen,1097
190,Zambia,1230


In [100]:
df8=pd.merge(df7,df4,how='inner',left_on=['Country/Region'],right_on=['COUNTRY'])
df8=df8.drop('COUNTRY',axis=1)
df8

,Country/Region,4/15/21,CODE
0,Afghanistan,2533,AFG
1,Albania,2335,ALB
2,Algeria,3144,DZA
3,Andorra,121,AND
4,Angola,557,AGO
...,...,...,...
182,Vietnam,35,VNM
183,West Bank and Gaza,2937,WBG
184,Yemen,1097,YEM
185,Zambia,1230,ZMB


In [101]:
# Check for missing values
empty= df8[df8.isna().any(axis=1)]
empty

,Country/Region,4/15/21,CODE


In [102]:
# Rename the Cases column
deathmapdata = df8.rename(columns={df8.columns[1]: 'Total Deaths'})
deathmapdata

,Country/Region,Total Deaths,CODE
0,Afghanistan,2533,AFG
1,Albania,2335,ALB
2,Algeria,3144,DZA
3,Andorra,121,AND
4,Angola,557,AGO
...,...,...,...
182,Vietnam,35,VNM
183,West Bank and Gaza,2937,WBG
184,Yemen,1097,YEM
185,Zambia,1230,ZMB


# 5. Fit the data to the Plotly Choropleth map

In [103]:
fig = go.Figure(data=go.Choropleth(
    locations = casemapdata['CODE'],
    z = deathmapdata['Total Deaths'],
    text = casemapdata['Country/Region'],
    colorscale = 'Blues',
    autocolorscale=False,
    reversescale=True,
    marker_line_color='darkgray',
    marker_line_width=0.5,
    colorbar_title = 'Total Deaths',
))

fig.update_layout(
    title_text='Cumulative Deaths per Country',
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    ),
    annotations = [dict(
        x=0.55,
        y=0.1,
        xref='paper',
        yref='paper',
        text='Source: <a href="https://github.com/CSSEGISandData/COVID-19">\
            JHU CSSE COVID-19 Data</a>',
        showarrow = False
    )]
)

fig.show()

In [104]:
deathmapdata.to_excel ("deathmapdata.xlsx", index = False, header=True)